Open Play Shots - Expected Behind Model - Model Tuning and Building - GBM

In [36]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/")

In [37]:
import pandas as pd
import numpy as np
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

from expected_score_model.config import open_shots_file_path
# from expected_score_model.domain.preprocessing.data_preprocessor import DataPreprocessor
# from expected_score_model.domain.contracts.mappings import Mappings
from expected_score_model.domain.contracts.modelling_data_contract import ModellingDataContract
from expected_score_model.domain.modelling.hyperparameter_tuning import XGBHyperparameterTuner
from expected_score_model.domain.modelling.supermodel import SuperXGBClassifier
from expected_score_model.domain.modelling.optuna_xgb_param_grid import OptunaXGBParamGrid

pd.options.display.max_rows = 100
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Model Version

In [38]:
model_version = 7
model_name = 'expected_behind_open'
model_file_name = model_name + '_v' + str(model_version)

model_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/models"
prediction_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/predictions/"

preprocessor_file_name = 'preprocessor_v' + str(model_version)
preprocessor_output_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/expected-score-model/model_outputs/preprocessors/"

In [39]:
RESPONSE = ModellingDataContract.RESPONSE_BEHIND

In [40]:
FEATURES = ModellingDataContract.feature_list_open_behind

In [41]:
MONOTONE_CONSTRAINTS = ModellingDataContract.monotone_constraints_open_behind

Load Data

In [42]:
df_modelling = pd.read_csv(open_shots_file_path)
df_modelling.tail()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Behind,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet
9211,211,kickIn,behind,1705.0,4,744,775.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,56.0,-36.0,ineffective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,0,1,0,1,0,1,107.0,32.0,75.0,Kick,Gather,Kick,Handball Received,56.0,48.0,7.0,4.0,-36.0,-44.0,-47.0,-37.0,775.0,771.0,769.0,766.0,31.0,4.0,11.313708,24.0,36.0,43.266615,0.982794,56.309932,0.082503,4.727050,0.258900,14.833859,576.0,3.178054,False,True,False,False,True,False,False,True,False,False
9212,212,kickIn,goal,1740.0,4,787,900.0,Geelong,Geelong,Brandan Parfitt,Brandan_Parfitt,Kick,75.0,0.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,113.0,33.0,80.0,Kick,Handball Received,Handball,Loose Ball Get,75.0,75.0,76.0,75.0,0.0,-1.0,-5.0,-6.0,900.0,899.0,898.0,897.0,113.0,1.0,1.000000,5.0,0.0,5.000000,0.000000,0.000000,2.168022,124.218492,-1.429422,-81.899864,25.0,1.609438,False,False,False,True,False,False,True,True,False,False
9213,218,possGain,goal,1837.0,4,1136,1198.0,Geelong,Geelong,Sam De Koning,Sam_De_Koning,Kick,78.0,1.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,119.0,39.0,80.0,Kick,Gather,Kick,Handball Received,78.0,78.0,65.0,64.0,1.0,1.0,-12.0,-9.0,1198.0,1193.0,1190.0,1188.0,62.0,5.0,0.000000,2.0,1.0,2.236068,0.463648,26.565051,-2.442748,-139.959156,-0.440569,-25.242748,4.0,0.693147,False,True,False,False,True,False,False,True,False,False
9214,227,throwIn,goal,1901.0,4,1452,1458.0,Geelong,Geelong,Joel Selwood,Joel_Selwood,Kick,37.0,17.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,6,0,125.0,45.0,80.0,Kick,Gather,Knock On,Handball,37.0,37.0,43.0,34.0,17.0,19.0,20.0,25.0,1458.0,1457.0,1455.0,1453.0,6.0,1.0,2.000000,43.0,17.0,46.238512,0.376490,21.571307,0.129338,7.410516,0.403551,23.121747,1849.0,3.761200,False,False,False,True,True,False,False,True,False,False
9215,233,possGain,goal,1971.0,4,1744,1753.0,Sydney,Sydney,Tom Papley,Tom_Papley,Kick,48.0,-20.0,effective,True,NaN,141,160,Geelong,Sydney,left,2022F4_Geelong_Sydney,2022F4,2022,1,0,0,6,0,6,137.0,45.0,92.0,Kick,Bounce,Handball Received,Handball,48.0,38.0,35.0,37.0,-20.0,-31.0,-33.0,-37.0,1753.0,1752.0,1748.0,1747.0,9.0,1.0,14.866069,32.0,20.0,37.735925,0.558599,32.005383,0.144862,8.299977,0.461317,26.431498,1024.0,3.465736,False,True,False,False,False,False,True,True,False,False


In [43]:
df_modelling[['ballUp', 'centreBounce', 'kickIn', 'possGain', 'throwIn']] = pd.get_dummies(df_modelling['Initial_State'])

In [44]:
training_data = df_modelling[df_modelling[RESPONSE+"TrainingSet"]]
test_data = df_modelling[df_modelling[RESPONSE+"TestSet"]]
cal_data = df_modelling[df_modelling[RESPONSE+"ValidationSet"]]

In [45]:
X_train, y_train = training_data.drop(columns=[RESPONSE]), training_data[RESPONSE]
X_test, y_test = test_data.drop(columns=[RESPONSE]), test_data[RESPONSE]
X_cal, y_cal = cal_data.drop(columns=[RESPONSE]), cal_data[RESPONSE]

In [46]:
X_test.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Goal,Miss,Score,Home_Score,Away_Score,Home_Current_Score,Away_Current_Score,Current_Margin,Event_Type0,Event_Type1,Event_Type2,Event_Type3,x0,x1,x2,x3,y0,y1,y2,y3,Quarter_Duration0,Quarter_Duration1,Quarter_Duration2,Quarter_Duration3,Chain_Duration,Time_Since_Last_Action,Distance_Since_Last_Action,Distance_to_Right_Goal_x,Distance_to_Middle_y,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Angle_to_Middle_Goal_degrees,Visible_Goal_Angle,Visible_Goal_Angle_degrees,Visible_Behind_Angle,Visible_Behind_Angle_degrees,Squared_Distance_to_Middle_Goal,Log_Distance_to_Middle_Goal,Set_Shot,GoalTrainingSet,GoalTestSet,GoalValidationSet,BehindTrainingSet,BehindTestSet,BehindValidationSet,MissTrainingSet,MissTestSet,MissValidationSet,ballUp,centreBounce,kickIn,possGain,throwIn
5,43,possGain,behind,444.0,1,1272,1317.0,Brisbane Lions,Brisbane Lions,Joe Daniher,Joe_Daniher,Kick,27.0,31.0,ineffective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,0,0,1,1,0,18.0,9.0,9.0,Kick,Handball Received,Handball,Free For,27.0,28.0,34.0,35.0,31.0,34.0,35.0,34.0,1317.0,1316.0,1314.0,1308.0,45.0,1.0,3.162278,51.0,31.0,59.682493,0.546167,31.293039,0.091898,5.265373,0.282203,16.169053,2601.0,3.931826,False,False,False,True,False,True,False,True,False,False,0,0,0,1,0
6,46,possGain,goal,476.0,1,1383,1389.0,Sydney,Sydney,Isaac Heeney,Isaac_Heeney,Kick,56.0,6.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,19.0,9.0,10.0,Kick,Handball Received,Handball,Handball Received,56.0,58.0,58.0,58.0,6.0,21.0,25.0,26.0,1389.0,1389.0,1388.0,1387.0,6.0,0.0,15.132746,22.0,6.0,22.803509,0.266252,15.255119,0.276208,15.825576,0.987285,56.567262,484.0,3.091042,False,True,False,False,False,True,False,False,True,False,0,0,0,1,0
8,79,possGain,goal,747.0,2,709,720.0,Sydney,Sydney,Sam Wicks,Sam_Wicks,Kick,41.0,18.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,0,6,20.0,40.0,-20.0,Kick,Handball Received,Handball,Handball Received,41.0,38.0,37.0,36.0,18.0,16.0,15.0,15.0,720.0,720.0,719.0,718.0,11.0,0.0,3.605551,37.0,18.0,41.146081,0.452778,25.942295,0.140721,8.062731,0.443767,25.425977,1369.0,3.610918,False,True,False,False,False,True,False,False,True,False,0,0,0,1,0
9,83,throwIn,goal,781.0,2,848,861.0,Brisbane Lions,Brisbane Lions,Jarryd Lyons,Jarryd_Lyons,Kick,46.0,2.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,6,0,20.0,46.0,-26.0,Kick,Gather,Knock On,Handball,46.0,41.0,35.0,30.0,2.0,2.0,-1.0,-2.0,861.0,861.0,860.0,858.0,13.0,0.0,5.000000,32.0,2.0,32.062439,0.062419,3.576334,0.201226,11.529413,0.656522,37.615967,1024.0,3.465736,False,True,False,False,False,True,False,True,False,False,0,0,0,0,1
12,95,possGain,goal,888.0,2,1285,1306.0,Brisbane Lions,Brisbane Lions,Keidean Coleman,Keidean_Coleman,Kick,48.0,0.0,effective,True,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021,1,0,6,6,0,32.0,46.0,-14.0,Kick,Loose Ball Get Crumb,Kick Inside 50 Result,Kick Into F50,48.0,45.0,51.0,31.0,0.0,-5.0,-8.0,-43.0,1306.0,1305.0,1303.0,1303.0,21.0,1.0,5.830952,30.0,0.0,30.000000,0.000000,0.000000,0.215789,12.363772,0.713012,40.852606,900.0,3.401197,False,True,False,False,False,True,False,True,False,False,0,0,0,1,0


Preprocess Data

In [47]:
# preprocessor = DataPreprocessor()
# preprocessor.fit(X_train)

# X_train_preproc = preprocessor.transform(X_train)
# X_test_preproc = preprocessor.transform(X_test)

In [48]:
X_train_preproc = X_train[FEATURES]
X_test_preproc = X_test[FEATURES]
X_cal_preproc = X_cal[FEATURES]

In [49]:
X_train_preproc.shape, X_test_preproc.shape

((5897, 6), (1844, 6))

In [50]:
X_train_preproc.head()

,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
0,26.0,-21.0,8.246211,56.080300,0.383818,0.106164
1,35.0,19.0,7.000000,47.010637,0.416065,0.125105
3,34.0,15.0,15.132746,46.486557,0.328553,0.130930
4,40.0,-11.0,4.123106,39.560081,0.281772,0.156423
7,65.0,-15.0,3.162278,19.849433,0.856706,0.216802


In [51]:
X_test_preproc.head()

,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
5,27.0,31.0,3.162278,59.682493,0.546167,0.091898
6,56.0,6.0,15.132746,22.803509,0.266252,0.276208
8,41.0,18.0,3.605551,41.146081,0.452778,0.140721
9,46.0,2.0,5.000000,32.062439,0.062419,0.201226
12,48.0,0.0,5.830952,30.000000,0.000000,0.215789


Optuna Hyperparameter Tuning Class - HyperParameterTuner & XGBHyperparameterTuner

In [52]:
xgb_tuner = XGBHyperparameterTuner(X_train_preproc, y_train, monotonicity_constraints=MONOTONE_CONSTRAINTS)

In [53]:
xgb_tuner.training_data.head()

,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle
0,26.0,-21.0,8.246211,56.080300,0.383818,0.106164
1,35.0,19.0,7.000000,47.010637,0.416065,0.125105
3,34.0,15.0,15.132746,46.486557,0.328553,0.130930
4,40.0,-11.0,4.123106,39.560081,0.281772,0.156423
7,65.0,-15.0,3.162278,19.849433,0.856706,0.216802


In [54]:
xgb_tuner.tune_hyperparameters()

[I 2023-07-11 15:48:07,932] A new study created in memory with name: no-name-0a436b18-6eab-4c20-9b90-9c880b06f18f
[I 2023-07-11 15:48:07,964] Trial 0 finished with value: 0.6754058173452518 and parameters: {'max_depth': 18, 'min_child_weight': 33, 'eta': 0.026870702860154822, 'gamma': 3.7093970366279945, 'lambda': 0.7892238937069453, 'alpha': 0.16564447434810015, 'subsample': 0.20448548127883415, 'colsample_bytree': 0.22207054685680433}. Best is trial 0 with value: 0.6754058173452518.
[I 2023-07-11 15:48:07,993] Trial 1 finished with value: 0.6816640983909799 and parameters: {'max_depth': 15, 'min_child_weight': 69, 'eta': 0.013647883314766303, 'gamma': 9.17039039697765, 'lambda': 1.8617197329451358, 'alpha': 0.03116708982711852, 'subsample': 0.502258707686315, 'colsample_bytree': 0.7667567652828389}. Best is trial 0 with value: 0.6754058173452518.
[I 2023-07-11 15:48:08,026] Trial 2 finished with value: 0.6752036948632543 and parameters: {'max_depth': 2, 'min_child_weight': 92, 'eta':

Number of finished trials:  1000
Best trial:
  Value: 0.6202148333504693
  Params: 
    max_depth: 6
    min_child_weight: 48
    eta: 0.3479220808361153
    gamma: 3.3623390554446724
    lambda: 0.007599871414143446
    alpha: 0.0001306474514313134
    subsample: 0.8013575648612602
    colsample_bytree: 0.8015764236649014


In [55]:
params = xgb_tuner.get_best_params()
params

{'max_depth': 6,
 'min_child_weight': 48,
 'eta': 0.3479220808361153,
 'gamma': 3.3623390554446724,
 'lambda': 0.007599871414143446,
 'alpha': 0.0001306474514313134,
 'subsample': 0.8013575648612602,
 'colsample_bytree': 0.8015764236649014}

Training Model - SuperXGBClassifier class for training and predictions

In [56]:
params['objective'] = OptunaXGBParamGrid.error
params['num_rounds'] = 1000
params['early_stopping_rounds'] = 50
params['verbosity'] = 1
params['monotone_constraints'] = MONOTONE_CONSTRAINTS

In [57]:
super_xgb = SuperXGBClassifier(X_train = X_train_preproc, 
                               y_train = y_train, 
                               X_test = X_test_preproc, 
                               y_test = y_test,
                               X_cal = X_cal_preproc,
                               y_cal = y_cal,
                               params = params)

In [58]:
super_xgb.fit()

[0]	validation_0-logloss:0.66422	validation_1-logloss:0.66384
[1]	validation_0-logloss:0.65069	validation_1-logloss:0.65040
[2]	validation_0-logloss:0.64385	validation_1-logloss:0.64324
[3]	validation_0-logloss:0.63986	validation_1-logloss:0.63836
[4]	validation_0-logloss:0.63873	validation_1-logloss:0.63716
[5]	validation_0-logloss:0.63803	validation_1-logloss:0.63638
[6]	validation_0-logloss:0.63737	validation_1-logloss:0.63553
[7]	validation_0-logloss:0.63640	validation_1-logloss:0.63552
[8]	validation_0-logloss:0.63587	validation_1-logloss:0.63575
[9]	validation_0-logloss:0.63525	validation_1-logloss:0.63546
[10]	validation_0-logloss:0.63526	validation_1-logloss:0.63546
[11]	validation_0-logloss:0.63434	validation_1-logloss:0.63523
[12]	validation_0-logloss:0.63338	validation_1-logloss:0.63512
[13]	validation_0-logloss:0.63338	validation_1-logloss:0.63512
[14]	validation_0-logloss:0.63336	validation_1-logloss:0.63512
[15]	validation_0-logloss:0.63292	validation_1-logloss:0.63503
[1

In [59]:
super_xgb.xgb_model

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8015764236649014, early_stopping_rounds=50,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=3.3623390554446724, gpu_id=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.34792...
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=48, missing=nan,
              monotone_constraints={'Angle_to_Middle_Goal': 1,
                                    'Distance_Since_Last_Action': -1,
                                    'Distance_to_Middle_Goal': 1,
                                    'Visible_Goal_Angle': -1, 'x0': 1},
              n_estimators=1000, n_jobs=None, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [60]:
super_xgb.xgb_model.get_booster().feature_names

['x0',
 'y0',
 'Distance_Since_Last_Action',
 'Distance_to_Middle_Goal',
 'Angle_to_Middle_Goal',
 'Visible_Goal_Angle']

In [61]:
train_preds = super_xgb.predict(X_train_preproc)
test_preds = super_xgb.predict(X_test_preproc)

In [62]:
train_probas = super_xgb.predict_proba(X_train_preproc)[:, 1]
test_probas = super_xgb.predict_proba(X_test_preproc)[:, 1]
cal_probas = super_xgb.predict_proba(X_cal_preproc)[:, 1]

In [63]:
super_xgb.calibrate()

In [64]:
train_cal_probas = super_xgb.predict_proba(X_train_preproc, calibrate=True)
test_cal_probas = super_xgb.predict_proba(X_test_preproc, calibrate=True)

Check Average Predictions

In [65]:
train_probas.mean(), training_data[RESPONSE].mean(), train_cal_probas.mean()

(0.36506873, 0.3647617432592844, 0.3660425748970197)

In [66]:
test_probas.mean(), test_data[RESPONSE].mean(), test_cal_probas.mean()

(0.364184, 0.36496746203904556, 0.36479400296932607)

Export model

In [67]:
super_xgb.export_model(model_output_path + "/" + model_file_name + ".joblib")

Export data and predictions

In [68]:
train_info = training_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
train_export = pd.concat([train_info, X_train_preproc.reset_index(drop=True)], axis=1)
train_export['xgb_preds'] = train_preds
train_export['xgb_probas'] = train_probas
train_export['xgb_probas_cal'] = train_cal_probas
train_export.to_csv(prediction_output_path + 'train_predictions_' + model_file_name + '.csv', index = False)
train_export.head()

,Match_ID,Chain_Number,Order,Behind,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,1,11.0,0,26.0,-21.0,8.246211,56.080300,0.383818,0.106164,1,0.512724,0.488981
1,202101_BrisbaneLions_Sydney,6,61.0,0,35.0,19.0,7.000000,47.010637,0.416065,0.125105,0,0.391180,0.395043
2,202101_BrisbaneLions_Sydney,21,222.0,0,34.0,15.0,15.132746,46.486557,0.328553,0.130930,0,0.412669,0.413227
3,202101_BrisbaneLions_Sydney,38,392.0,0,40.0,-11.0,4.123106,39.560081,0.281772,0.156423,0,0.386451,0.390947
4,202101_BrisbaneLions_Sydney,50,512.0,1,65.0,-15.0,3.162278,19.849433,0.856706,0.216802,0,0.417339,0.417085


In [69]:
test_info = test_data[['Match_ID', "Chain_Number", "Order", RESPONSE]].reset_index(drop = True)
test_export = pd.concat([test_info, X_test_preproc.reset_index(drop=True)], axis=1)
test_export['xgb_preds'] = test_preds
test_export['xgb_probas'] = test_probas
test_export['xgb_probas_cal'] = test_cal_probas
test_export.to_csv(prediction_output_path + 'test_predictions_' + model_file_name + '.csv', index = False)
test_export.head()

,Match_ID,Chain_Number,Order,Behind,x0,y0,Distance_Since_Last_Action,Distance_to_Middle_Goal,Angle_to_Middle_Goal,Visible_Goal_Angle,xgb_preds,xgb_probas,xgb_probas_cal
0,202101_BrisbaneLions_Sydney,43,444.0,1,27.0,31.0,3.162278,59.682493,0.546167,0.091898,0,0.455573,0.447452
1,202101_BrisbaneLions_Sydney,46,476.0,0,56.0,6.0,15.132746,22.803509,0.266252,0.276208,0,0.298295,0.308112
2,202101_BrisbaneLions_Sydney,79,747.0,0,41.0,18.0,3.605551,41.146081,0.452778,0.140721,0,0.417562,0.417268
3,202101_BrisbaneLions_Sydney,83,781.0,0,46.0,2.0,5.000000,32.062439,0.062419,0.201226,0,0.320398,0.330036
4,202101_BrisbaneLions_Sydney,95,888.0,0,48.0,0.0,5.830952,30.000000,0.000000,0.215789,0,0.326945,0.336382


Save preprocessor

In [ ]:
# joblib.dump(preprocessor, preprocessor_output_path + preprocessor_file_name + ".joblib")